In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('Social_Network_Ads.csv')
data.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [17]:
X = data.iloc[:,[2,3]].values
y = data.iloc[:,4].values

In [18]:
data.shape

(400, 5)

In [19]:
len(X)

400

In [20]:
len(y)

400

In [21]:
y

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1,

In [8]:
X[0:5]

array([[   19, 19000],
       [   35, 20000],
       [   26, 43000],
       [   27, 57000],
       [   19, 76000]], dtype=int64)

In [9]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)

In [23]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [24]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [25]:
X_train[0:5]

array([[ 0.01833971, -0.00469012],
       [-0.45596314,  1.22646747],
       [-0.36110257, -0.82546185],
       [-0.64568428,  0.52294884],
       [ 0.39778199, -0.0340034 ]])

In [26]:
from sklearn.ensemble import RandomForestClassifier

In [27]:
classifier = RandomForestClassifier(
    n_estimators=10, #default 100
    criterion='entropy', #default 'gini'
    random_state=10
)

In [32]:
classifier.fit(X_train,y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=10)

In [37]:
y_pred = classifier.predict(X_test)

In [34]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [35]:
confusion_matrix(y_test,y_pred)

array([[61,  6],
       [ 4, 29]], dtype=int64)

In [36]:
accuracy_score(y_test,y_pred)

0.9

In [28]:
from sklearn.model_selection import cross_val_score

In [29]:
cross_val = cross_val_score(
    classifier,
    X,
    y,
    cv=10,
    scoring='accuracy').mean()

In [30]:
cross_val

0.8775000000000001

In [38]:
from sklearn.model_selection import RandomizedSearchCV

In [39]:
model = RandomForestClassifier(n_jobs=-1)

In [40]:
from scipy.stats import randint

In [41]:
hyperparameters = {
    'max_depth': [3,5,10,None],
    'n_estimators': [10, 100, 200, 300, 400],
    'max_features': randint(1,3),
    'criterion': ['gini', 'entropy'],
    'bootstrap': [True, False],
    'min_samples_leaf': randint(1,4),
}

In [44]:
def hypertuning_random_forest(model, hyperparameters, n_iterations, X, y):
    random_search = RandomizedSearchCV(
        model, 
        param_distributions=hyperparameters,
        n_jobs=-1,
        n_iter = n_iterations,
        cv = 9
    )
    random_search.fit(X,y)
    ht_params = random_search.best_params_
    ht_score = random_search.best_score_
    return ht_params, ht_score

In [45]:
rf_parameters, rf_ht_score = hypertuning_random_forest(
    model,
    hyperparameters,
    40,
    X,
    y
)

In [46]:
rf_parameters

{'bootstrap': True,
 'criterion': 'entropy',
 'max_depth': 3,
 'max_features': 2,
 'min_samples_leaf': 1,
 'n_estimators': 100}

In [47]:
rf_ht_score

0.9120089786756453

In [48]:
classifier = RandomForestClassifier(
    n_jobs=-1,
    n_estimators=300,
    bootstrap=True,
    criterion='entropy',
    max_depth=3,
    max_features=2,
    min_samples_leaf=1
)

In [50]:
cross_val_score(classifier,X,y,cv=10,scoring='accuracy').mean()

0.8875

In [51]:
classifier.fit(X_train,y_train)

RandomForestClassifier(criterion='entropy', max_depth=3, max_features=2,
                       n_estimators=300, n_jobs=-1)

In [52]:
y_pred = classifier.predict(X_test)

In [53]:
confusion_matrix(y_test, y_pred)

array([[59,  8],
       [ 2, 31]], dtype=int64)

In [54]:
accuracy_score(y_test, y_pred)

0.9